In [1]:
from IPython.display import clear_output

!pip install telebot torch torchvision
clear_output()

In [27]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [28]:
!cp drive/'My Drive'/ViT_B_16_weights.pth .

In [29]:
import os
import tempfile
import cv2
import telebot
from PIL import Image
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights

In [30]:
id2label={0:'battery', 1: 'biological', 2:'brown-glass', 3:'cardboard', 4:'green-glass', 5:'metal', 6:'paper', 7:'plastic', 8:'trash', 9:'white-glass'}

In [31]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

num_classes = 10  # Replace with your actual number of classes

# Load the pre-trained Vision Transformer model
model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
model.heads.head = torch.nn.Linear(model.heads.head.in_features, num_classes)

# Load the state dictionary into the model
model_path = 'ViT_B_16_weights.pth'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()
clear_output()

In [32]:
def garbage_class_recognition(image):
    with torch.no_grad():
        # Perform the forward pass
        outputs = model(image)
        # Get the predicted class index
        _, predicted = torch.max(outputs, 1)
    return predicted.item()

In [33]:
# link: https://t.me/Garbage_Classifier_bot

API_TOKEN = '7209626391:AAGTcSQ0hSPI_m90FKvg7wgX_QYJPkZBydg'
bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Hi, I'm a bot assistant in sorting garbage! Send me an image of a separate item of garbage and I will tell you which class of waste it belongs to.")

@bot.message_handler(content_types=['photo'])
def handle_image(message):

    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as temp_file:
        temp_file.write(downloaded_file)
        temp_file_path = temp_file.name

    image = cv2.imread(temp_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)

    image = transform(image).unsqueeze(0)

    category = garbage_class_recognition(image)
    name=id2label[int(category)]

    bot.reply_to(message, f'Recognized category:\n{name}')

    os.remove(temp_file_path)

bot.polling()